In [1]:
%load_ext autoreload
%autoreload 2

In [36]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [37]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")
df

,rides_t-168,rides_t-167,rides_t-166,rides_t-165,rides_t-164,rides_t-163,rides_t-162,rides_t-161,rides_t-160,rides_t-159,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id,target
0,4,42,9,40,76,65,30,4,40,73,...,106,60,25,3,28,37,49,2024-01-28 20:00:00,5626,40
1,42,9,40,76,65,30,4,40,73,68,...,60,25,3,28,37,49,40,2024-01-29 00:00:00,5626,10
2,9,40,76,65,30,4,40,73,68,87,...,25,3,28,37,49,40,10,2024-01-29 04:00:00,5626,25
3,40,76,65,30,4,40,73,68,87,43,...,3,28,37,49,40,10,25,2024-01-29 08:00:00,5626,89
4,76,65,30,4,40,73,68,87,43,9,...,28,37,49,40,10,25,89,2024-01-29 12:00:00,5626,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10140,32,137,111,133,51,9,31,137,108,147,...,8,13,52,118,88,45,14,2024-12-31 04:00:00,6072,8
10141,137,111,133,51,9,31,137,108,147,48,...,13,52,118,88,45,14,8,2024-12-31 08:00:00,6072,80
10142,111,133,51,9,31,137,108,147,48,5,...,52,118,88,45,14,8,80,2024-12-31 12:00:00,6072,105
10143,133,51,9,31,137,108,147,48,5,27,...,118,88,45,14,8,80,105,2024-12-31 16:00:00,6072,63


In [38]:
from datetime import datetime

from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2024, 10, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(7385, 170)
(7385,)
(2760, 170)
(2760,)


In [39]:
import numpy as np

class BaselineModelPreviousHour:

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test["rides_t-1"]

In [40]:
model = BaselineModelPreviousHour()
predictions = model.predict(X_test)

In [41]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

79.9496


In [42]:
!pip install dagshub



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [43]:
from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow
from dotenv import load_dotenv
load_dotenv() 
import os
mlflow = set_mlflow_tracking()

INFO:httpx:HTTP Request: GET https://dagshub.com/api/v1/repos/mnv12699/citibike "HTTP/1.1 200 OK"


Initialized MLflow to track repo "mnv12699/citibike"

INFO:dagshub:Initialized MLflow to track repo "mnv12699/citibike"


Repository mnv12699/citibike initialized!

INFO:dagshub:Repository mnv12699/citibike initialized!
INFO:src.experiment_utils:MLflow tracking URI and credentials set.


In [44]:
%pip install -q dagshub mlflow


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [45]:
import dagshub
import mlflow
dagshub.init(repo_owner='mnv12699', repo_name='citibike', mlflow=True)

# Set the MLflow Tracking URI via Dagshub initialization
print("✅ MLflow Tracking URI:", mlflow.get_tracking_uri())

INFO:httpx:HTTP Request: GET https://dagshub.com/api/v1/repos/mnv12699/citibike "HTTP/1.1 200 OK"


Initialized MLflow to track repo "mnv12699/citibike"

INFO:dagshub:Initialized MLflow to track repo "mnv12699/citibike"


Repository mnv12699/citibike initialized!

INFO:dagshub:Repository mnv12699/citibike initialized!


✅ MLflow Tracking URI: https://dagshub.com/mnv12699/citibike.mlflow


In [46]:
from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow
from dotenv import load_dotenv
load_dotenv() 
import os
mlflow = set_mlflow_tracking()

INFO:httpx:HTTP Request: GET https://dagshub.com/api/v1/repos/mnv12699/citibike "HTTP/1.1 200 OK"


Initialized MLflow to track repo "mnv12699/citibike"

INFO:dagshub:Initialized MLflow to track repo "mnv12699/citibike"


Repository mnv12699/citibike initialized!

INFO:dagshub:Repository mnv12699/citibike initialized!
INFO:src.experiment_utils:MLflow tracking URI and credentials set.


In [47]:
log_model_to_mlflow(model, X_test, "BaselineModelPreviousHour", "mean_absolute_error", score=test_mae)

INFO:src.experiment_utils:Experiment set to: BaselineModelPreviousHour
INFO:src.experiment_utils:Logged mean_absolute_error: 79.94963768115942
/opt/anaconda3/envs/my_proj_cda500_env/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning:

Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.

INFO:src.experiment_utils:Model signature inferred.
Registered model 'BaselineModelPreviousH

🏃 View run burly-doe-768 at: https://dagshub.com/mnv12699/citibike.mlflow/#/experiments/0/runs/62ebbe88c4974d3191c50427719d0528
🧪 View experiment at: https://dagshub.com/mnv12699/citibike.mlflow/#/experiments/0


In [48]:
import numpy as np

class BaselineModelPreviousWeek:

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test[f"rides_t-{7*6}"]

In [49]:
model = BaselineModelPreviousWeek()
predictions = model.predict(X_test)

In [50]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

32.4547


In [51]:
log_model_to_mlflow(model, X_test, "BaselineModelPreviousWeek", "mean_absolute_error", score=test_mae)

INFO:src.experiment_utils:Experiment set to: BaselineModelPreviousWeek
INFO:src.experiment_utils:Logged mean_absolute_error: 32.45471014492754
/opt/anaconda3/envs/my_proj_cda500_env/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning:

Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.

INFO:src.experiment_utils:Model signature inferred.
Registered model 'BaselineModelPreviousW

🏃 View run rebellious-roo-460 at: https://dagshub.com/mnv12699/citibike.mlflow/#/experiments/1/runs/1f6063f0d52a4848970d563dfe778756
🧪 View experiment at: https://dagshub.com/mnv12699/citibike.mlflow/#/experiments/1


In [53]:
import numpy as np
import pandas as pd

class BaselineModelLast4Weeks:
    """
    A baseline model that predicts the average of the last 4 weeks (28 days)
    for each test instance.
    """

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        """
        The fit method is not used in this baseline model as it does not learn
        from the training data.
        """
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        """
        Predicts the average of the last 4 weeks (28 days) for each test instance.

        Parameters:
            X_test (pd.DataFrame): The test DataFrame containing lagged features
                                   (e.g., rides_t-{7*24}, rides_t-{14*24}, etc.).

        Returns:
            np.array: An array of predictions based on the average of the last 4 weeks.
        """
        # Define the columns for the last 4 weeks
        last_4_weeks_columns = [
            f"rides_t-{7*6}",  # 1 week ago
            f"rides_t-{14*6}", # 2 weeks ago
            f"rides_t-{21*6}", # 3 weeks ago
            f"rides_t-{28*6}"  # 4 weeks ago
        ]

        # Ensure the required columns exist in the test DataFrame
        for col in last_4_weeks_columns:
            if col not in X_test.columns:
                raise ValueError(f"Missing required column: {col}")

        # Calculate the average of the last 4 weeks
        predictions = X_test[last_4_weeks_columns].mean(axis=1)

        return predictions.to_numpy()

In [54]:
model = BaselineModelLast4Weeks()
predictions = model.predict(X_test)

In [55]:
X_test[X_test['pickup_location_id']==5626] 

,rides_t-168,rides_t-167,rides_t-166,rides_t-165,rides_t-164,rides_t-163,rides_t-162,rides_t-161,rides_t-160,rides_t-159,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
0,25,91,254,188,279,133,36,98,245,172,...,212,110,35,82,214,211,252,112,2024-10-01 00:00:00,5626
1,91,254,188,279,133,36,98,245,172,276,...,110,35,82,214,211,252,112,26,2024-10-01 04:00:00,5626
2,254,188,279,133,36,98,245,172,276,184,...,35,82,214,211,252,112,26,105,2024-10-01 08:00:00,5626
3,188,279,133,36,98,245,172,276,184,31,...,82,214,211,252,112,26,105,232,2024-10-01 12:00:00,5626
4,279,133,36,98,245,172,276,184,31,106,...,214,211,252,112,26,105,232,180,2024-10-01 16:00:00,5626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,69,146,127,178,92,13,66,177,125,158,...,55,13,21,96,157,120,82,35,2024-12-31 04:00:00,5626
548,146,127,178,92,13,66,177,125,158,70,...,13,21,96,157,120,82,35,22,2024-12-31 08:00:00,5626
549,127,178,92,13,66,177,125,158,70,8,...,21,96,157,120,82,35,22,78,2024-12-31 12:00:00,5626
550,178,92,13,66,177,125,158,70,8,55,...,96,157,120,82,35,22,78,145,2024-12-31 16:00:00,5626


In [56]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

31.2624


In [57]:
log_model_to_mlflow(model, X_test, "BaselineModelLast4Weeks", "mean_absolute_error", score=test_mae)

INFO:src.experiment_utils:Experiment set to: BaselineModelLast4Weeks
INFO:src.experiment_utils:Logged mean_absolute_error: 31.262409420289856
/opt/anaconda3/envs/my_proj_cda500_env/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning:

Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.

INFO:src.experiment_utils:Model signature inferred.
Registered model 'BaselineModelLast4Weeks

🏃 View run gifted-wasp-129 at: https://dagshub.com/mnv12699/citibike.mlflow/#/experiments/2/runs/5892f5f8f5584d8a9291209a08f47f35
🧪 View experiment at: https://dagshub.com/mnv12699/citibike.mlflow/#/experiments/2


In [58]:
from src.plot_utils import plot_aggregated_time_series

plot_aggregated_time_series(X_test, y_test, 200, predictions)